# Gravitational potential and gravity attraction - Prism

**Reference: Blakely (1996)**

In [8]:
# Importing all usefull libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

In [9]:
# Positions and number of points
n = 500
x = np.linspace(-3000, 3000, n) # meters
y = np.copy(x)
z = -100. # meters
# Computes the mesh of all points
X, Y = np.meshgrid(x, y)

In [10]:
# Setting the parameters for prism
model = [-1000., 1000., -1000., 1000., 50., 100., 2800] # meters + g/cm3
# set area for plot the rectangle over the contourmap:
area = [ model[0], model[1], model[2], model[3] ]

In [11]:
from modules.prism import prism_gx, prism_gy, prism_gz
from modules.plots import prism3D, rectangle

In [12]:
prism3D(model, 45, 30, 'x(m)', 'y(m)', 'Depth(m)' ,'black', 'model')

In [13]:
# Calculating the gravity components
gx = prism_gx(X, Y, z, model)
gy = prism_gy(X, Y, z, model)
gz = prism_gz(X, Y, z, model)

In [14]:
# Ploting all results 
plt.close('all')
plt.figure(figsize=(20,14))

plt.subplot(2, 2, 1)
plt.title('Gravity $X$ component (mGal)')
plt.contourf(X, Y, gx, 20, cmap = plt.cm.jet)
plt.colorbar()
rectangle(area, style='-k', linewidth=2, fill=None, alpha=1., label=None)
plt.xlabel('X coordinate (m)')
plt.ylabel('Y coordinate (m)')


plt.subplot(2, 2, 2)
plt.title('Gravity $Y$ component (mGal)')
plt.contourf(X, Y, gy, 20, cmap = plt.cm.jet)
plt.colorbar()
rectangle(area, style='-k', linewidth=2, fill=None, alpha=1., label=None)
plt.xlabel('X coordinate (m)')
plt.ylabel('Y coordinate (m)')

plt.subplot(2, 2, 3)
plt.title('Gravity $Z$ component (mGal)')
plt.contourf(X, Y, gz, 20, cmap = plt.cm.jet)
plt.colorbar()
rectangle(area, style='-k', linewidth=2, fill= None, alpha=0.5, label='Prism')
plt.xlabel('X coordinate (m)')
plt.ylabel('Y coordinate (m)')

#plt.savefig('figures/prism-gravity')

plt.show()